# ハイパーパラメーターのチューニング

*ハイパーパラメーター* (トレーニングに影響を与えるが、トレーニングデータ自体からは決定できないパラメーター値) を必要とする機械学習アルゴリズムは多数あります。たとえば、ロジスティック回帰モデルをトレーニングする場合、*正規化率* ハイパーパラメーターを使用してモデルのバイアスに対抗できます。畳み込みニューラル ネットワークをトレーニングする場合、*学習率* や *バッチ サイズ* などのハイパーパラメーターを使用して、重みの調整方法とミニバッチで処理されるデータ項目の数をそれぞれ制御できます。ハイパーパラメーター値の選択は、トレーニング済みモデルのパフォーマンスやトレーニングにかかる時間に大きく影響する可能性があります。多くの場合、最適なソリューションを見つけるには複数の組み合わせを試す必要があります。

この場合、単一のハイパーパラメーターを使用したロジスティック回帰モデルの簡単な例を使用しますが、Azure Machine Learning でトレーニングできるあらゆる種類のモデルに原則が適用されます。

## ワークスペースに接続する

まず、Azure ML SDK を使用してワークスペースに接続する必要があります。

> **注**: 前回の演習を完了してから Azure サブスクリプションとの認証済みセッションの有効期限が切れている場合は、再認証を求めるメッセージが表示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存した構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 実験用データを準備する

このラボでは、糖尿病患者の詳細を含むデータセットを使用します。次のセルを実行してこのデータセットを作成します (前のラボで作成した場合、コードは新しいバージョンを作成します)。

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

## トレーニング スクリプトを準備する

まず、ロジスティック回帰モデルのトレーニングに使用するトレーニング スクリプト用フォルダーを作成します。

In [ ]:
import os

experiment_folder = 'diabetes_training-hyperdrive'
os.makedirs(experiment_folder, exist_ok=True)

print('Folder ready.')

ここで、モデルをトレーニングする Python スクリプトを作成します。これには、次の項目が含まれている必要があります。

- 最適化する各ハイパーパラメーターのパラメーター (この場合は、正規化ハイパーパラメーターのみ)
- 最適化するパフォーマンス メトリックを記録するコード (この場合、AUC と精度の両方を記録するため、どちらかのモデルを最適化することを選択できます)

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# ライブラリをインポートする
import argparse
import joblib
import os
from azureml.core import Run
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 正規化パラメーターを設定する
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument("--input-data", type=str, dest='input_data', help='training dataset')
args = parser.parse_args()
reg = args.reg_rate

# 実験実行コンテキストを取得する
run = Run.get_context()

# 糖尿病データセットを読み込む
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe() # Get the training data from the estimator input

# 特徴とラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# ロジスティック回帰モデルのトレーニング
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 正確さを計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 出力フォルダーに保存されたファイルは、自動的に実験レコードにアップロードされます
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## コンピューティング先を準備する

クラウド コンピューティングの利点の 1 つは、オンデマンドでスケーリングできることです。これにより、それぞれが異なるハイパーパラメーター値を持つ実験の複数の実行を並行して処理するのに十分なコンピューティング リソースをプロビジョニングできます。

以前のラボで作成した Azure Machine Learning コンピューティング クラスターを使用します (これがない場合は作成されます)。

> **重要**: 実行する前に、*コンピューティング クラスター* を以下のコードでコンピューティング クラスターの名前に変更してください。クラスター名は、長さが 2 〜 16 文字のグローバルに一意の名前である必要があります。英字、数字、- の文字が有効です。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

## *ハイパードライブ* 実験を実行する

Azure Machine Learning には、*ハイパードライブ* 実験を通じたハイパーパラメーター チューニング機能が含まれています。これらの実験は、それぞれが異なるハイパーパラメーターの組み合わせで、複数の子の実行を起動します。最適なモデルを作成する実行 (最適化するターゲットのパフォーマンス メトリックの記録によって決定される) を特定し、登録およびデプロイに選択されたトレーニング済みモデルを特定できます。

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import GridParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice
from azureml.widgets import RunDetails

# 実験用 Python 環境を作成する
sklearn_env = Environment("sklearn-env")

# 必要なパッケージがインストールされていることを確認する (scikit-learn、Azure ML defaults、Azure ML dataprep が必要)
packages = CondaDependencies.create(pip_packages=['scikit-learn','azureml-defaults','azureml-dataprep[pandas]'])
sklearn_env.python.conda_dependencies = packages

# トレーニング データセットを取得する
diabetes_ds = ws.datasets.get("diabetes dataset")

# スクリプト構成を作成する
script_config = ScriptRunConfig(source_directory=experiment_folder,
                              script='diabetes_training.py',
                              arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                           '--input-data', diabetes_ds.as_named_input('training_data')], # Reference to dataset
                              environment=sklearn_env,
                              compute_target = training_cluster)

# パラメーター値の範囲をサンプリングする
params = GridParameterSampling(
    {
        # There's only one parameter, so grid sampling will try each value - with multiple parameters it would try every combination
        '--regularization': choice(0.001, 0.005, 0.01, 0.05, 0.1, 1.0)
    }
)

# ハイパードライブ設定を構成する
hyperdrive = HyperDriveConfig(run_config=script_config, 
                          hyperparameter_sampling=params, 
                          policy=None, 
                          primary_metric_name='AUC', 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=6,
                          max_concurrent_runs=4)

# 実験を実行する
experiment = Experiment(workspace = ws, name = 'diabates_training_hyperdrive')
run = experiment.submit(config=hyperdrive)

# 実験の実行時に Notebook の状態を表示する
RunDetails(run).show()
run.wait_for_completion()

上記のウィジェットで実験の実行状態を表示できます。また、[Azure Machine Learning Studio](https://ml.azure.com) で、メインのハイパードライブ実験の実行とその子の実行を表示することもできます。

> **注**: ウィジェットは更新されない場合があります。実行が完了すると、ウィジェットの下に概要情報が表示されます。

## パフォーマンスの最も高い実行を決定する

すべての実行が終了したら、指定したパフォーマンス メトリック (この場合は最高の AUC を持つもの) に基づいて最適な実行を見つけることができます。

In [ ]:
for child_run in run.get_children_sorted_by_primary_metric():
    print(child_run)

best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details() ['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print(' -AUC:', best_run_metrics['AUC'])
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Regularization Rate:',parameter_values)

最適な実行が見つかったので、トレーニングしたモデルを登録できます。

In [ ]:
from azureml.core import Model

# モデルを登録する
best_run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                        tags={'Training context':'Hyperdrive'},
                        properties={'AUC': best_run_metrics['AUC'], 'Accuracy': best_run_metrics['Accuracy']})

# 登録済みモデルを一覧表示する
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **詳細情報**: ハイパードライブの詳細については、[Azure ML のドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-tune-hyperparameters)を参照してください。